## Q7. Graph Convolutional Network (GCN)

Graph Convolutional Networks (GCNs) are similar to convolutions in images in the sense that the "filter" parameters are typically shared over all locations in the graph. At the same time, GCNs rely on message passing methods, which means that vertices exchange information with the neighbors, and send "messages" to each other. Before looking at the math, we can try to visually understand how GCNs work. 

- The first step is that each node creates a feature vector that represents the message it wants to send to all its neighbors. 

- In the second step, the messages are sent to the neighbors, so that a node receives one message per adjacent node. Below we have visualized the two steps for our example graph. 

<center width="100%" style="padding:10px"><img src="graph_message_passing.svg" width="700px"></center>

If we want to formulate that in more mathematical terms, we need to first decide how to combine all the messages a node receives. As the number of messages vary across nodes, we need an operation that works for any number. Hence, the usual way to go is to sum or take the mean. Given the previous features of nodes $H^{(l)}$, the GCN layer is defined as follows:

$$H^{(l+1)} = \sigma\left(\hat{D}^{-1/2}\hat{A}\hat{D}^{-1/2}H^{(l)}W^{(l)}\right)$$

$W^{(l)}$ is the weight parameters with which we transform the input features into messages ($H^{(l)}W^{(l)}$). To the adjacency matrix $A$ we add the identity matrix so that each node sends its own message also to itself: $\hat{A}=A+I$. Finally, to take the average instead of summing, we calculate the matrix $\hat{D}$ which is a diagonal matrix with $D_{ii}$ denoting the number of neighbors node $i$ has. $\sigma$ represents an arbitrary activation function, and not necessarily the sigmoid (usually a ReLU-based activation function is used in GNNs). 

When implementing the GCN layer in PyTorch, we can take advantage of the flexible operations on tensors. Instead of defining a matrix $\hat{D}$, we can simply divide the summed messages by the number of neighbors afterward. Additionally, we replace the weight matrix with a linear layer, which additionally allows us to add a bias. Written as a PyTorch module, the GCN layer is defined as follows:

In [2]:
import torch
import torch.nn as nn

class GCNLayer(nn.Module):
    
    def __init__(self, c_in, c_out):
        super().__init__()
        self.projection = nn.Linear(c_in, c_out)

    def forward(self, node_feats, adj_matrix):
        """
        Inputs:
            node_feats - Tensor with node features of shape [batch_size, num_nodes, c_in]
            adj_matrix - Batch of adjacency matrices of the graph. If there is an edge from i to j, adj_matrix[b,i,j]=1 else 0.
                         Supports directed edges by non-symmetric matrices. Assumes to already have added the identity connections. 
                         Shape: [batch_size, num_nodes, num_nodes]
        """
        
        #===============================================================
        # TODO: Calculate the number of neighbors for each node
        degree_matrix = torch.sum(adj_matrix, dim=-1)
        degree_matrix_inv_sqrt = torch.diag_embed(1.0 / torch.sqrt(degree_matrix + 1e-6))
        # TODO: Apply a linear projection to the node features
        norm_adj_matrix = torch.matmul(degree_matrix_inv_sqrt, adj_matrix)
        norm_adj_matrix = torch.matmul(norm_adj_matrix, degree_matrix_inv_sqrt)
        # TODO: Aggregate neighbor features for each node
        aggregated_feats = torch.matmul(norm_adj_matrix, node_feats)
        # TODO: Normalize aggregated features by the number of neighbors
        node_feats = self.projection(aggregated_feats)

        # 第2步：对邻接矩阵进行标准化
   

        # 第3步：聚合邻居特征
        

        # 第4步：对节点特征应用线性投影


        #===============================================================

        return node_feats

To further understand the GCN layer, we can apply it to our example graph above. First, let's specify some node features and the adjacency matrix with added self-connections:

In [3]:
node_feats = torch.arange(8, dtype=torch.float32).view(1, 4, 2)

# TODO: Define the adjacency matrix in the graph above with self-connections
adj_matrix =  torch.ones(1, 4, 4) 

print("Node features:\n", node_feats)
print("\nAdjacency matrix:\n", adj_matrix)

Node features:
 tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])

Adjacency matrix:
 tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])


Next, let's apply a GCN layer to it. For simplicity, we initialize the linear weight matrix as an identity matrix so that the input features are equal to the messages. This makes it easier for us to verify the message passing operation.

In [4]:
layer = GCNLayer(c_in=2, c_out=2)
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])

with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix)

print("Input features", node_feats)
print("Output features", out_feats)

Input features tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])
Output features tensor([[[3.0000, 4.0000],
         [3.0000, 4.0000],
         [3.0000, 4.0000],
         [3.0000, 4.0000]]])
